In [33]:
import os
log_dir = '/vol/vssp/mammo2/will/logs/new'
fCount = 0
while os.path.exists(os.path.join(log_dir, 'training', str(fCount))):
    fCount+=1
    
print(os.path.exists(os.path.join(log_dir, 'training')))
print(log_dir + 'hello')

True
/vol/vssp/mammo2/will/logs/newhello


In [1]:
# fold
import cv2
import glob
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
# For some reason I have to tell it to use TensorFlows dimension ordering
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.callbacks import TensorBoard
from time import time
import os
import tensorflow as tf
from keras import applications

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/vol/vssp/mammo2/will/logs/new', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        fCount=0
        while os.path.exists(os.path.join(log_dir, 'training' + '_' + str(fCount))):
            fCount+=1
        training_log_dir = os.path.join(log_dir, 'training' + '_' + str(fCount))
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation' + '_' + str(fCount))

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


def get_images(path, dataSpecs):
    fileList = glob.glob(path) #'BengaliBMPConvert/*.bmp'   
    num = len(fileList)
    dataSpecs['classLength'].append(len(fileList))
    x = np.array([(cv2.imread(fname)) for fname in fileList])
    return x

def get_labels_one_hot(num_classes, class_id, num_samples):
    x = np.zeros((num_samples, num_classes))
    x[np.arange(num_samples),class_id] = 1
    return x

def fourCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    #model.add(Conv2D(32, (3,3), activation='relu'))   
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model
    
def fiveCNN():
    model = Sequential()
    # Layer 1
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(385, 385, 1), data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 2
    model.add(Conv2D(32, (3,3), activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 3    
    model.add(Conv2D(32, (3,3), activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 4     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    # Layer 5     
    model.add(Conv2D(32, (3,3), activation='relu'))   
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Layer 6
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))  
    return model
    
    
def main():
    import keras
    print('keras version: ', keras.__version__)
    print('TensorFlow version: ', tf.__version__)
    print('\nLoad images...')
    
    # Get images and labels
    data = {'img': 0, 'label': 0}
    dataSpecs = {'classLength': []}
    dataSpecs['classLength'] = []
    data['img'] = np.concatenate((
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/0/*', dataSpecs), # Class 0
            #get_images('/vol/vssp/mammo2/will/data/simulated/calcs/small_sample/1/*', dataSpecs) # Class 1
            get_images('/user/HS204/wm0015/student/allCalcs/0/*', dataSpecs), # Class 0
            get_images('/user/HS204/wm0015/student/allCalcs/1/*', dataSpecs) # Class 1
    ))      

    # Normalise
    data['img'] = data['img']/255    
    # Print image    
    img_calc = data['img']   
    plt.imshow(img_calc[0], cmap='gray')
    plt.show()
    
    # Create one hot labels
    labels_bg = get_labels_one_hot(2, 0, dataSpecs['classLength'][0])  
    labels_calc = get_labels_one_hot(2, 1, dataSpecs['classLength'][1])
    data['label'] = np.concatenate((
            get_labels_one_hot(2, 0, dataSpecs['classLength'][0]), # Class 0 
            get_labels_one_hot(2, 1, dataSpecs['classLength'][1]) # Class 1
    ))
    # Drop from 3 colour channels to 1 (greyscale)
    if 1==0:
        data['img'] = data['img'][:,:,:,0]
        data['img'] = np.reshape(data['img'], (data['img'].shape[0],data['img'].shape[1],data['img'].shape[2],1))
        print('new data shape = ', data['img'].shape)
    
    valStats = []
    folds = 10
    for crossVal in range(folds):

        # Shuffle data
        seed = 33
    #     np.random.seed(seed) # Has to be set before each use of random
        shuffleMask = np.random.permutation(data['img'].shape[0])    
        data['img'] = data['img'][shuffleMask, :, :, :]
        data['label'] = data['label'][shuffleMask, :]

        # Split traing and validation data        
        splitRatio = 0.9
        splitPoint = math.floor(data['img'].shape[0]*splitRatio)
        train_data = {'img': data['img'][0:splitPoint], 'label': data['label'][0:splitPoint]}
        val_data = {'img': data['img'][splitPoint:], 'label': data['label'][splitPoint:]}
#         print('train_data[img].shape = ', train_data['img'].shape)
#         print('train_data[label].shape = ', train_data['label'].shape)
#         print('val_data[img].shape = ', val_data['img'].shape)
#         print('valdiation_data[label].shape = ', val_data['label'].shape)
#         print('dataSpecs = ', dataSpecs['classLength'])

        print('Compute bottleneck features...')
        vggModel = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (385, 385, 3))
        # Freeze all layers
        for layer in vggModel.layers:
            layer.trainable = False
        # Add custom final layer
        bNModel = vggModel.output
        bNModel = Flatten()(bNModel)
        final_model = keras.models.Model(inputs=vggModel.input, outputs=bNModel)
        train_bNFeatures = {'img': 0, 'label': train_data['label']}
        val_bNFeatures = {'img': 0, 'label': val_data['label']}
        train_bNFeatures['img'] = final_model.predict(train_data['img'], batch_size=16)
        val_bNFeatures['img'] = final_model.predict(val_data['img'], batch_size=16)
        print('train_bNFeatures[img].shape = ', train_bNFeatures['img'].shape)
        print('val_bNFeatures[img].shape = ', val_bNFeatures['img'].shape)
        
        print('Train head...')
        head = Sequential()
        head.add(Dense(32, input_dim=train_bNFeatures['img'].shape[1], activation='relu'))
        head.add(Dense(2, activation='softmax'))
        
        sgd = optimizers.SGD(lr=5, decay=1e-6, momentum=0.9, nesterov=True)
        adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #0.001
        head.compile(loss='binary_crossentropy',
                    optimizer=adam,
                     metrics=['accuracy'])
        tensorboard = TensorBoard(log_dir='/vol/vssp/mammo2/will/logs/new'.format(time()), write_images=True)
        if folds == 1:
            head.summary()  

        # Train
        head.fit(train_bNFeatures['img'], train_data['label'], 
                batch_size=100, epochs=150, verbose=1,
                callbacks=[TrainValTensorBoard(write_graph=False)],
                validation_data=(val_bNFeatures['img'], val_bNFeatures['label']))

        # Evaluate
        score = head.evaluate(val_bNFeatures['img'], val_bNFeatures['label'], verbose=0)
        valStats.append(score)
        print('The score is......\n', score)
        
#         model_final = keras.Model(input = model.input, output = customVgg)
        

# #         model.compile(loss='binary_crossentropy',
# #                 optimizer=adam,
# #                 metrics=['accuracy']) 
  

    valStats = np.asarray(valStats)
    print('Validations: \n', valStats[:, 1])
    print('Average loss: ', sum(valStats[:, 0])/folds)
    print('Average validation: ', sum(valStats[:, 1])/folds)
    
if __name__ == "__main__":
    main()

Using TensorFlow backend.


keras version:  2.2.0
TensorFlow version:  1.7.1

Load images...


<Figure size 640x480 with 1 Axes>

Compute bottleneck features...
train_bNFeatures[img].shape =  (1458, 73728)
val_bNFeatures[img].shape =  (162, 73728)
Train head...
Train on 1458 samples, validate on 162 samples
Epoch 1/150
1458/1458 [==============================] - 1s 879us/step - loss: 1.5988 - acc: 0.4492 - val_loss: 0.6937 - val_acc: 0.3827
Epoch 2/150
1458/1458 [==============================] - 1s 347us/step - loss: 0.6933 - acc: 0.4506 - val_loss: 0.6931 - val_acc: 0.6173
Epoch 3/150
1458/1458 [==============================] - 0s 307us/step - loss: 0.6930 - acc: 0.5494 - val_loss: 0.6921 - val_acc: 0.6173
Epoch 4/150
1458/1458 [==============================] - 0s 326us/step - loss: 0.6925 - acc: 0.5494 - val_loss: 0.6910 - val_acc: 0.6173
Epoch 5/150
1458/1458 [==============================] - 0s 275us/step - loss: 0.6921 - acc: 0.5494 - val_loss: 0.6897 - val_acc: 0.6173
Epoch 6/150
1458/1458 [==============================] - 0s 330us/step - loss: 0.6916 - acc: 0.5494 - val_loss: 0.6887 - val_acc: 0.6173

1458/1458 [==============================] - 1s 456us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6748 - val_acc: 0.6173
Epoch 60/150
1458/1458 [==============================] - 0s 327us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6747 - val_acc: 0.6173
Epoch 61/150
1458/1458 [==============================] - 0s 287us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6747 - val_acc: 0.6173
Epoch 62/150
1458/1458 [==============================] - 0s 306us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6748 - val_acc: 0.6173
Epoch 63/150
1458/1458 [==============================] - 1s 350us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6748 - val_acc: 0.6173
Epoch 64/150
1458/1458 [==============================] - 0s 286us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6747 - val_acc: 0.6173
Epoch 65/150
1458/1458 [==============================] - 1s 345us/step - loss: 0.6883 - acc: 0.5494 - val_loss: 0.6750 - val_acc: 0.6173
Epoch 66/150
1458/1458 [=======================

...

1458/1458 [==============================] - 0s 327us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 26/150
1458/1458 [==============================] - 0s 323us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 27/150
1458/1458 [==============================] - 0s 280us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 28/150
1458/1458 [==============================] - 0s 334us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 29/150
1458/1458 [==============================] - 1s 374us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 30/150
1458/1458 [==============================] - 1s 351us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 31/150
1458/1458 [==============================] - 0s 292us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 32/150
1458/1458 [=======================

1458/1458 [==============================] - 1s 360us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 85/150
1458/1458 [==============================] - 0s 321us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 86/150
1458/1458 [==============================] - 0s 328us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 87/150
1458/1458 [==============================] - 0s 331us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 88/150
1458/1458 [==============================] - 0s 325us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 89/150
1458/1458 [==============================] - 0s 303us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 90/150
1458/1458 [==============================] - 0s 296us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 91/150
1458/1458 [=======================

1458/1458 [==============================] - 1s 344us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 144/150
1458/1458 [==============================] - 0s 273us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 145/150
1458/1458 [==============================] - 0s 296us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 146/150
1458/1458 [==============================] - 0s 245us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 147/150
1458/1458 [==============================] - 0s 298us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 148/150
1458/1458 [==============================] - 0s 272us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 149/150
1458/1458 [==============================] - 1s 353us/step - loss: 8.8727 - acc: 0.4465 - val_loss: 9.3015 - val_acc: 0.4198
Epoch 150/150
1458/1458 [================

Epoch 51/150
1458/1458 [==============================] - 0s 247us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 52/150
1458/1458 [==============================] - 0s 309us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 53/150
1458/1458 [==============================] - 0s 295us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 54/150
1458/1458 [==============================] - 0s 283us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 55/150
1458/1458 [==============================] - 0s 324us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 56/150
1458/1458 [==============================] - 0s 260us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 57/150
1458/1458 [==============================] - 0s 248us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 58/150
1458/1458 [==========

1458/1458 [==============================] - 0s 323us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 111/150
1458/1458 [==============================] - 0s 304us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 112/150
1458/1458 [==============================] - 1s 356us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 113/150
1458/1458 [==============================] - 1s 413us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 114/150
1458/1458 [==============================] - 0s 319us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 115/150
1458/1458 [==============================] - 1s 354us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 116/150
1458/1458 [==============================] - 0s 296us/step - loss: 8.8837 - acc: 0.4458 - val_loss: 9.2025 - val_acc: 0.4259
Epoch 117/150
1458/1458 [================

In [2]:
print('hello world')

hello world


In [12]:
x = [[0.6940992025681484, 0.3950617283950617], [0.6884314866713536, 0.5185185185185185], [0.6855149121932042, 0.5679012345679012]]
print(x[0])
print(sum(x[:][1])/2)

[0.6940992025681484, 0.3950617283950617]
0.603475002594936


In [35]:
x = [[6.5308382717179665, 0.5925925925925926], [0.16470135086112553, 0.9444444444444444], [0.15422181794304907, 0.9444444444444444], [0.19875195217721256, 0.9382716049382716]] 
x = np.asarray(x)
print(x, '\n')
print(x[0][0])
print(x[:, 0])
print(x.shape)

[[6.53083827 0.59259259]
 [0.16470135 0.94444444]
 [0.15422182 0.94444444]
 [0.19875195 0.9382716 ]] 

6.5308382717179665
[6.53083827 0.16470135 0.15422182 0.19875195]
(4, 2)
